<a href="https://colab.research.google.com/github/Unalunapark/practice/blob/main/5_2_%EC%9E%91%EC%97%85%ED%98%952_%EB%AA%A8%EC%9D%98%EA%B3%A0%EC%82%AC2_%EC%97%90%EC%96%B4%EB%B9%84%EC%97%94%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 에어비엔비 가격?
- 평가: R-Squared, MAE, MSE, RMSE, RMSLE, MAPE
- target : price(가격)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
id,price
34323697,238
29927138,183
120362,234
~~~



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 불러오기

In [ ]:
import pandas as pd
test=pd.read_csv("/content/drive/MyDrive/데이터모음/딴짓/abnyc/test.csv")
train=pd.read_csv("/content/drive/MyDrive/데이터모음/딴짓/abnyc/train.csv")
train.info() #price
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39116 entries, 0 to 39115
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              39116 non-null  int64  
 1   name                            39102 non-null  object 
 2   host_id                         39116 non-null  int64  
 3   host_name                       39100 non-null  object 
 4   neighbourhood_group             39116 non-null  object 
 5   neighbourhood                   39116 non-null  object 
 6   latitude                        39116 non-null  float64
 7   longitude                       39116 non-null  float64
 8   room_type                       39116 non-null  object 
 9   minimum_nights                  39116 non-null  int64  
 10  number_of_reviews               39116 non-null  int64  
 11  last_review                     31063 non-null  object 
 12  reviews_per_month               

##EDA

## ! 문제 분석
- 수치형
- 문자형

In [ ]:
ntrain=train.select_dtypes(exclude='object')
ntest=test.select_dtypes(exclude='object')

In [ ]:
ntrain=ntrain.fillna(0)
ntest=ntest.fillna(0)

In [ ]:
ntrain.isnull().sum() #name, host_name, last_review, reviews_per_month

id                                0
host_id                           0
latitude                          0
longitude                         0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
price                             0
dtype: int64

In [ ]:
ntest.isnull().sum() #name, host_name, last_review, reviews_per_month

id                                0
host_id                           0
latitude                          0
longitude                         0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [ ]:
# # help(pd.to_datetime)
# train['last_review']=pd.to_datetime(train['last_review'])
# test['ltas']
# train['last_review'].info()

In [ ]:
#  **Handling various input formats**

#     Assembling a datetime from multiple columns of a :class:`DataFrame`. The keys
#     can be common abbreviations like ['year', 'month', 'day', 'minute', 'second',
#     'ms', 'us', 'ns']) or plurals of the same

#     >>> df = pd.DataFrame({'year': [2015, 2016],
#     ...                    'month': [2, 3],
#     ...                    'day': [4, 5]})
#     >>> pd.to_datetime(df)
#     0   2015-02-04
#     1   2016-03-05
#     dtype: datetime64[ns]


KeyError: 'ltas'

In [ ]:
# train['year']=train['last_review'].dt.year
# train['month']=train['last_review'].dt.month
# train['day']=train['last_review'].dt.day

## 데이터 전처리 & 피처엔지니어링

In [ ]:
ntrain.describe()

,id,host_id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price
count,3.911600e+04,3.911600e+04,39116.000000,39116.000000,39116.000000,39116.000000,39116.000000,39116.000000,39116.000000,39116.000000
mean,1.896669e+07,6.737070e+07,40.729054,-73.952084,7.022395,23.268509,1.090076,7.206182,112.718478,153.057547
std,1.096743e+07,7.842425e+07,0.054451,0.046106,19.547706,44.625410,1.598609,33.100783,131.479798,248.758522
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,9.435824e+06,7.829861e+06,40.690250,-73.983060,1.000000,1.000000,0.040000,1.000000,0.000000,69.000000
50%,1.961906e+07,3.050966e+07,40.723060,-73.955680,3.000000,5.000000,0.370000,1.000000,45.000000,105.000000
75%,2.907327e+07,1.074344e+08,40.763130,-73.936017,5.000000,23.000000,1.572500,2.000000,225.000000,175.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,999.000000,607.000000,58.500000,327.000000,365.000000,10000.000000


## 검증 데이터 분리

In [ ]:
# import sklearn.model_selection
# help(sklearn.model_selection.train_test_split)

In [ ]:
from sklearn.model_selection import train_test_split
xtr, xval, ytr, yval=train_test_split(ntrain.drop('price',axis=1), ntrain['price'], test_size=0.1, random_state=2024)
xtr.shape, xval.shape, ytr.shape, yval.shape

((35204, 9), (3912, 9), (35204,), (3912,))

In [ ]:
xtr.head()

,id,host_id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
23954,18240963,114736959,40.57609,-73.98605,3,52,1.95,6,150
9008,28890908,192270861,40.81993,-73.94768,5,1,0.19,1,0
17282,10116220,51913826,40.72354,-73.99333,1,1,0.30,8,0
21083,21943047,29238030,40.71284,-73.94805,2,3,0.20,1,0
15725,11981509,25237492,40.84216,-73.94101,30,7,0.19,34,322


In [ ]:
xval.head()

,id,host_id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
14219,9778579,50411330,40.71504,-73.98879,1,2,0.05,1,0
35050,16738648,24626134,40.75329,-73.98880,2,1,0.03,1,0
15718,23771352,25705033,40.70604,-73.93802,2,1,0.07,1,0
17208,27190005,204733512,40.76087,-73.88375,3,19,1.67,2,0
7383,6042635,31358851,40.85882,-73.92897,4,38,0.79,1,307


In [ ]:
ytr.head()

23954    110
9008      75
17282     84
21083     70
15725     95
Name: price, dtype: int64

In [ ]:
yval.head()

14219     45
35050     30
15718    190
17208     45
7383      65
Name: price, dtype: int64

## 모델 & 평가

In [ ]:
# 평가
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
def rmse(y_test, y_pred): #RMSE
    return np.sqrt(mean_squared_error(y_test, y_pred))

def rmsle(y_test, y_pred): #RMSLE
    return np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))

def mape(y_test, y_pred): #MAPE
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

## 예측 및 csv 제출

##정리

In [ ]:
# y_test = pd.read_csv("y_test.csv")
# print(r2_score(y_test, pred))